# 책추천 협업필터링

# 데이터 불러옴

In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 40.8 MB/s eta 0:00:00


In [4]:
!pip install sshtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.7 MB/s eta 0:00:00


In [6]:
!pip install "paramiko<3.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: paramiko
    Found existing installation: paramiko 4.0.0
    Uninstalling paramiko-4.0.0:
      Successfully uninstalled paramiko-4.0.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

path = '/content/drive/MyDrive/recommend_system/'


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

# 실제 접속은 포함하지 않음 — 샘플 형식만 유지
def get_connection():
    """
    🔒 서버 보안 보호를 위해 공개 레포에서는 실제 연결 로직을 제거했습니다.
    """
    pass


SSH 터널 연결됨


In [9]:
query = """
SELECT
    r.user_id,
    r.book_id,
    r.rating,
    b.title,
    b.author,
    b.category_name,
    b.cover
FROM reviews r
JOIN books b ON r.book_id = b.book_id
WHERE r.rating IS NOT NULL
  AND r.book_id IS NOT NULL;
"""

df = pd.read_sql(query, conn)
conn.close()

  df = pd.read_sql(query, conn)



In [12]:
print(df.tail(5))

    user_id  book_id  rating             title                     author  \
25       30       25       4  나미야 잡화점의 기적 (양장)  히가시노 게이고 (지은이), 양윤옥 (옮긴이)   
26       14       24       5               데미안    헤르만 헤세 (지은이), 전영애 (옮긴이)   
27       13       31       5              코스모스     칼 세이건 (지은이), 홍승수 (옮긴이)   
28       34       32       5       자소서 바이블 3.0                  이준희 (지은이)   
29       14       35       3      소스 코드: 더 비기닝     빌 게이츠 (지은이), 안진환 (옮긴이)   

                        category_name  \
25   국내도서>소설/시/희곡>일본소설>1950년대 이후 일본소설   
26                  국내도서>소설/시/희곡>독일소설   
27                   국내도서>과학>천문학>우주과학   
28  국내도서>수험서/자격증>취업/상식/적성>취업 면접/자기소개서   
29     국내도서>경제경영>기업/경영자 스토리>국외 기업/경영자   

                                                cover  
25  https://image.aladin.co.kr/product/2175/33/cov...  
26  https://image.aladin.co.kr/product/26/0/covers...  
27  https://image.aladin.co.kr/product/54/7/covers...  
28  https://image.aladin.co.kr/product/37021/58/co...  
29  https:

### 협업필터링용 메트릭스 구성

In [26]:
user_item_matrix = df.pivot_table(index='user_id', columns='book_id', values='rating')
matrix_filled = user_item_matrix.fillna(0)

### 코사인 유사도 계산

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# =======================================
# 3️⃣ 유저 간 유사도 계산 (Cosine Similarity)
# =======================================
user_similarity = cosine_similarity(matrix_filled)
user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

print("\n✅ 유저 간 유사도 행렬 (상위 5개):")
print(user_similarity_df.head())



✅ 유저 간 유사도 행렬 (상위 5개):
user_id        12        13        14   30   31   34
user_id                                             
12       1.000000  0.702782  0.321356  0.0  0.0  0.0
13       0.702782  1.000000  0.205768  0.0  0.0  0.0
14       0.321356  0.205768  1.000000  0.0  0.0  0.0
30       0.000000  0.000000  0.000000  1.0  0.0  0.0
31       0.000000  0.000000  0.000000  0.0  1.0  0.0


In [28]:
# =======================================
# 4️⃣ 유저 기반 협업 필터링 추천 함수
# =======================================
def recommend_books_for_user(user_id, df, user_item_matrix, user_similarity_df, top_n=5):
    """
    특정 user_id가 아직 읽지 않은 책 중에서
    유사한 사용자의 평점을 기반으로 추천
    """
    if user_id not in user_similarity_df.index:
        print(f"❌ user_id {user_id}는 데이터에 없습니다.")
        return None

    # 1️⃣ 입력 유저와 다른 유저들 간 유사도
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]  # 자기 자신 제외

    # 2️⃣ 각 책에 대해 예측 평점 계산 (유사도 가중 평균)
    weighted_ratings = np.dot(similar_users.values, matrix_filled.loc[similar_users.index])
    sim_sums = similar_users.sum()
    predicted_ratings = weighted_ratings / (sim_sums + 1e-9)

    # 3️⃣ 이미 본 책 제외
    rated_books = matrix_filled.loc[user_id][matrix_filled.loc[user_id] > 0].index
    predicted_ratings = pd.Series(predicted_ratings, index=matrix_filled.columns)
    predicted_ratings = predicted_ratings.drop(rated_books, errors='ignore')

    # 4️⃣ 상위 N개 책 선택
    top_books = predicted_ratings.sort_values(ascending=False).head(top_n)

    # 5️⃣ 원본 df에서 책 정보 매핑
    recommended = df[df['book_id'].isin(top_books.index)][
        ['book_id', 'title', 'author', 'category_name', 'cover']
    ].drop_duplicates().reset_index(drop=True)
    recommended['predicted_rating'] = recommended['book_id'].map(top_books)

    return recommended.sort_values(by='predicted_rating', ascending=False)

In [30]:
# =======================================
# 5️⃣ 테스트: 특정 user_id 추천
# =======================================
target_user = 12  # 예시
recommendations = recommend_books_for_user(target_user, df, user_item_matrix, user_similarity_df, top_n=5)

print(f"\n📚 User {target_user} 추천 결과:")
print(recommendations[['book_id', 'title', 'author', 'predicted_rating', 'cover']])


📚 User 12 추천 결과:
   book_id                        title  \
1       15  애기 햄스터 애햄이 - 저학년 이상 창작동화 11   
4       31                         코스모스   
0       10   도쿄 커피명가에서 진심으로 알려주는 카페 디저트   
2       16      해리 포터와 불의 잔 (인터랙티브 에디션)   
3       24                          데미안   

                                       author  predicted_rating  \
1                           안선모 (글), 정임복 (그림)           3.43109   
4                      칼 세이건 (지은이), 홍승수 (옮긴이)           3.43109   
0           다구치 후미코, 다구치 마모루 (지은이), 임지인 (옮긴이)           1.56891   
2  J.K. 롤링 (지은이), 칼 제임스 파운트퍼드 (그림), 강동혁 (옮긴이)           1.56891   
3                     헤르만 헤세 (지은이), 전영애 (옮긴이)           1.56891   

                                               cover  
1  https://image.aladin.co.kr/product/39/44/cover...  
4  https://image.aladin.co.kr/product/54/7/covers...  
0  https://image.aladin.co.kr/product/37077/46/co...  
2  https://image.aladin.co.kr/product/37296/99/co...  
3  https://image.aladin.co.kr/product/26/0/co